<a href="https://colab.research.google.com/github/F1owly/EmotionDetector/blob/main/Emotions_detection_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files #Для загрузки своей картинки
import numpy as np #Библиотека работы с массивами
import pandas as pd
from PIL import Image
from matplotlib import image
from matplotlib import pyplot as plt
import random #Для генерации случайных чисел
import math # Для округления
import os #Для работы с файлами

from tensorflow.keras.models import Sequential, load_model #Сеть прямого распространения
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization #свертки и пр.
from tensorflow.keras.optimizers import Adam, Adadelta # оптимизаторы

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Загрузка данных

In [ ]:
our_size = (244, 244)

In [ ]:
path = '/content/drive/MyDrive/data/Angry'
X_angry = []
for dirs,folder,files in os.walk(path):
  for f in files:
    img = Image.open(path + '/' + f).resize(our_size).convert('RGB')
    #img.show()
    for i in range(0, 360, 10):
      X_angry.append(np.asarray(img.rotate(i)))

In [ ]:
path = '/content/drive/MyDrive/data/Happy'
X_happy = []
for dirs,folder,files in os.walk(path):
  for f in files:
    img = Image.open(path + '/' + f).resize(our_size).convert('RGB')
    for i in range(0, 360, 10):
      X_happy.append(np.asarray(img.rotate(i)))

In [ ]:
path = '/content/drive/MyDrive/data/Sad'
X_sad = []
for dirs,folder,files in os.walk(path):
  for f in files:
    img = Image.open(path + '/' + f).resize(our_size).convert('RGB')
    for i in range(0, 360, 10):
      X_sad.append(np.asarray(img.rotate(i)))

In [ ]:
len(X_angry), len(X_happy), len(X_sad)

(3096, 3600, 2844)

In [ ]:
X_angry = np.array(X_angry)
X_happy = np.array(X_happy)
X_sad = np.array(X_sad)

In [ ]:
len(X_angry)

3096

#Подготовка данных для обучения

In [ ]:
y_angry = X_angry.shape[0] * [[1, 0, 0]]
y_happy = X_happy.shape[0] * [[0, 1, 0]]
y_sad = X_sad.shape[0] * [[0, 0, 1]]

In [ ]:
y_angry[0]

[1, 0, 0]

In [ ]:
from sklearn.model_selection import train_test_split

def train_test_val_split(X,
                         y,
                         test_size = 0.2,
                         val_size = 0.1,
                         shuffle=True,
                         random_state=42):
  X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=test_size,
                                                    shuffle=shuffle,
                                                    random_state=random_state)

  X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=val_size/(1-test_size),
                                                    shuffle=shuffle,
                                                    random_state=random_state)
  return X_train, X_test, X_val, y_train, y_test, y_val


X_angry_train, X_angry_test, X_angry_val, y_angry_train, y_angry_test, y_angry_val = train_test_val_split(X_angry,
                                                                                                          y_angry,
                                                                                                          test_size = 0.2,
                                                                                                          val_size = 0.1,
                                                                                                          shuffle=True,
                                                                                                          random_state=42)

X_sad_train, X_sad_test, X_sad_val, y_sad_train, y_sad_test, y_sad_val = train_test_val_split(            X_sad,
                                                                                                          y_sad,
                                                                                                          test_size = 0.2,
                                                                                                          val_size = 0.1,
                                                                                                          shuffle=True,
                                                                                                          random_state=42)

X_happy_train, X_happy_test, X_happy_val, y_happy_train, y_happy_test, y_happy_val = train_test_val_split(X_happy,
                                                                                                          y_happy,
                                                                                                          test_size = 0.2,
                                                                                                          val_size = 0.1,
                                                                                                          shuffle=True,
                                                                                                          random_state=42)



X_train = np.concatenate([X_angry_train, X_sad_train, X_happy_train])
X_test = np.concatenate([X_angry_test, X_sad_test, X_happy_test])
X_val = np.concatenate([X_angry_val, X_sad_val, X_happy_val])
y_train = np.concatenate([y_angry_train, y_sad_train, y_happy_train])
y_test = np.concatenate([y_angry_test, y_sad_test, y_happy_test])
y_val = np.concatenate([y_angry_val, y_sad_val, y_happy_val])

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((6676, 244, 244, 3),
 (6676, 3),
 (1909, 244, 244, 3),
 (1909, 3),
 (955, 244, 244, 3),
 (955, 3))

In [ ]:
batch_size = 128
input_shape = (our_size[0], our_size[1], 3)

#Обучение нейросети

In [ ]:
#Создаем последовательную модель
model = Sequential()

# 32 Conv 3 + 32 Conv3 + MaxPool2
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


# 64 Conv 3 + 64 Conv3 + MaxPool2
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


# 128 Conv 3 + 128 Conv3 + 128 Conv3 + MaxPool2
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


# 128 Conv 3 + 128 Conv3 + 128 Conv3 + MaxPool2
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


# 256 Conv 3 + 256 Conv3 + 256 Conv3 + MaxPool2
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dense(1024, activation='relu'))

model.add(Dense(1024, activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(3, activation='softmax'))


model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["f1_score"])

In [ ]:
history = model.fit(X_train,
                    y_train,
                    batch_size=batch_size,
                    epochs=16,
                    validation_data=(X_val, y_val),
                    verbose=1)

#Отображаем график точности обучения
plt.plot(history.history['f1_score'],
         label='F1-score на обучающем наборе')
plt.plot(history.history['val_f1_score'],
         label='F1-score на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.save("/content/drive/MyDrive/Emotion_detector/my_model2.h5")

#Тестировка

In [ ]:
model = load_model("/content/drive/MyDrive/Emotion_detector/my_model2.h5")

In [ ]:
model.summary()

In [ ]:
scores = model.evaluate(X_test, y_test)
scores

#Демонстрация работы